<a href="https://colab.research.google.com/github/ailunguo/Test/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras

In [2]:
from tensorflow.keras.layers import CenterCrop
from tensorflow.keras.layers import Rescaling

In [3]:
from tensorflow.keras import layers

In [4]:
class CustomModel(keras.Model):
  def train_step(self, data):
    x, y = data
    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)
      loss = self.compute_loss(y=y, y_pred=y_pred)
    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    for metric in self.metrics:
      if metric.name == 'loss':
        metric.update_state(loss)
      else:
        metric.update_state(y, y_pred)
    return {m.name: m.result() for m in self.metrics}

inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=[...])

model.fit(dataset, epochs=3, callbacks=...)

NameError: ignored

# 使用多个GPU加速训练

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = Model()
  model.compile()

trian_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

model.evaluate(test_dataset)

# 在设备上同步进行预处理与在主机CPU上异步进行预处理

### 在CPU上进行异步操作, 只需将dataset.map预处理操作注入到数据管道中

In [8]:
from keras.src.layers.preprocessing.text_vectorization import TextVectorization
samples = np.array([['This is the 1st sample.'], ["And here's the 2nd sample."]])
labels = [[0], [1]]

vectorizer = TextVectorization(output_mode='int')
vectorizer.adapt(samples)

dataset = tf.data.Dataset.from_tensor_slices((samples, labels)).batch(2)

dataset = dataset.map(lambda x, y: (vectorizer(x), y))

dataset = dataset.prefetch(2)

inputs = keras.Input(shape=(None,), dtype='int64')
x = layers.Embedding(input_dim=10, output_dim=32)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='mse', run_eagerly=True)
model.fit(dataset)


1/1 [==============================] - 0s 309ms/step - loss: 0.5282


In [10]:
dataset = tf.data.Dataset.from_tensor_slices((samples, labels)).batch(2)

inputs = keras.Input(shape=(1,), dtype='string')
x = vectorizer(inputs)
x = layers.Embedding(input_dim=10, output_dim=32)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer='adam', loss='mse', run_eagerly=True)
model.fit(dataset)

1/1 [==============================] - 0s 444ms/step - loss: 0.5138
